In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
class Value:
      
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): 
    return self * other

  def __truediv__(self, other):
    return self * other**-1

  def __neg__(self): 
    return self * -1

  def __sub__(self, other): 
    return self + (-other)

  def __radd__(self, other): 
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  def relu(self):
    x = self.data
    r = max(0, x)
    out = Value(r, (self, ), 'relu')
    
    def _backward():
      self.grad += (x > 0) * out.grad
    out._backward = _backward
    
    return out
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad 
    out._backward = _backward
    
    return out
  def __gt__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    return self.data > other.data

  def log(self):
    x = self.data
    out = Value(math.log(x), (self, ), 'log')
    
    def _backward():
      self.grad += (1/x) * out.grad
    out._backward = _backward
    
    return out
  
  def sigmoid(self):
    x = self.data
    s = 1/(1 + math.exp(-x))
    out = Value(s, (self, ), 'sigmoid')
    
    def _backward():
      self.grad += s * (1 - s) * out.grad
    out._backward = _backward
    
    return out
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [3]:
class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(0,1)) for _ in range(nin)]
    self.b = Value(random.uniform(0,1))
  
  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:

  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]
  



In [4]:
def mse(y, ypred):
        out = Value(0)
        for i in range(len(ypred)):
            out += (ypred[i] - y[i]) ** 2
        out = out / Value(len(ypred))
        return out
    
def softmax(ypred):
    out = [v.exp() for v in ypred]
    s = sum(out)
    return [v/s for v in out]

def crossEntropy(y, ypred):
    ypred = softmax(ypred)
    out = Value(0)
    for i in range(len(ypred)):
        out += y[i] * ypred[i].log() + (1-y[i]) * (Value(1) - ypred[i]).log()
    return Value(-1/len(ypred)) * out
# the padding to keep the size of the input and output the same


In [13]:
class Conv2D : 
    def __init__(self, kernel_size, stride, num_filters, padding = 0, bias = False):
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.num_filters = num_filters
        
        # self.kernels = [
        #     [[Value(random.uniform(0, 1)) for _ in range(kernel_size)] for _ in range(kernel_size)]
        #     for _ in range(num_filters)
        # ]
        self.kernels = [[[Value(0), Value(-1), Value(0)], [Value(-1), Value(4), Value(-1)], [Value(0), Value(-1), Value(0)]]]
        if bias:
            self.bias = [Value(random.uniform(0,1)) for _ in range(num_filters)]
        else:
            self.bias = [Value(0) for _ in range(num_filters)]
            
    def __call__(self, x):
        out = []
        for i in range(self.num_filters):
            out.append(self.convop(x, self.kernels[i], self.bias[i]))
            
        return out

    def sliding_window(self, x):
        out = []
        if self.padding:
            x = [[Value(0) for _ in range(self.padding)] + row + [Value(0) for _ in range(self.padding)] for row in x]
            x = [[Value(0) for _ in range(len(x[0]))] for _ in range(self.padding)] + x + [[Value(0) for _ in range(len(x[0]))] for _ in range(self.padding)]
        for i in range(0, len(x) - self.kernel_size + 1, self.stride):  
            for j in range(0, len(x[0]) - self.kernel_size + 1, self.stride):  
                out.append([[x[k][l] for l in range(j, j + self.kernel_size)] for k in range(i, i + self.kernel_size)])
        return out

    
    def convop(self, x, kernel, bias):
        out  = []

        sliding_window = self.sliding_window(x)
        output_size = (len(x) - self.kernel_size + 2 * self.padding)/self.stride  + 1
        out_list = []
        for window in sliding_window:
            dot_prod = sum([window[i][j] * kernel[i][j] for i in range(self.kernel_size) for j in range(self.kernel_size)])
            res = dot_prod + bias
            out.append(res)
            if len(out) == output_size:
                out_list.append(out)
                out = []
            
        return out_list
            
    def Relu(self, x):
        for i in range(len(x)):
            for j in range(len(x[0])):
                for k in range(len(x[i][j])):
                    x[i][j][k] = x[i][j][k].relu()
        return x
        

def MaxPooling(x, pool_size, stride):
    out = []
    
    for i in range(0, len(x) - pool_size + 1, stride):  
        row = []
        for j in range(0, len(x[0]) - pool_size + 1, stride):  
            window = [x[k][l] for k in range(i, i + pool_size) for l in range(j, j + pool_size)]
            row.append(max(window))
        out.append(row)
    
    return out 
  
x = Conv2D(2, 1, 1)
input_image = [
    [Value(1), Value(2), Value(3)],
    [Value(4), Value(5), Value(6)],
    [Value(7), Value(8), Value(9)]
]

# res = x(input_image)
# print(x.Relu(res))
# MaxPooling(input_image, 2, 1)




In [20]:
from PIL import Image
import numpy as np

def load_image(image_path, resize_to=None):
    img = Image.open(image_path).convert('L')  
    if resize_to:
        img = img.resize(resize_to)
    img_array = np.array(img)  
    return img_array

def image_to_value_array(image_array):
    return [[Value(float(pixel)) for pixel in row] for row in image_array]

def value_array_to_image(value_array):
    return np.array([[val.data for val in row] for row in value_array])

def save_image(output_array, output_path):
    output_image = Image.fromarray(np.uint8(output_array))  # Convert back to image
    output_image.save(output_path)

def test_on_image(image_path, output_path, kernel_size=3, stride=1):
    img_array = load_image(image_path)
    print(f"Original image size: {img_array.shape}")
    input_image = image_to_value_array(img_array)
    
    conv_layer = Conv2D(kernel_size=kernel_size, stride=stride, num_filters=1)
    conv_result = conv_layer(input_image)
    relu_result = conv_layer.Relu(conv_result)

    output_array = value_array_to_image(relu_result[0])  
    
    output_array = (output_array - output_array.min()) / (output_array.max() - output_array.min()) * 255
    
    save_image(output_array, output_path)
    print(f"Processed image saved as {output_path}")

test_on_image("image.jpeg", "output1.png", kernel_size=3, stride=1)


Original image size: (225, 225)
Processed image saved as output1.png
